### Foodie-Talk Agentic RAG
This notebook builds out the LangGraph graph to help answer the user's food specofic questions. 

In [40]:
import os
import tiktoken
from typing import Any, Callable, List, Optional, TypedDict, Union

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client.http.models import Distance, VectorParams
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain_community.tools import TavilySearchResults


from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

from langgraph.graph import START, END, StateGraph

In [23]:
search = TavilySearchResults(max_results=5)
search.invoke("Pizza in Manhattan")

[{'title': 'BEST of Manhattan, NY Pizza - Yelp',
  'url': 'https://www.yelp.com/search?find_desc=Pizza&find_loc=Manhattan%2C+NY',
  'content': "Top 10 Best Pizza Near Manhattan, New York · 1. Joe's Pizza · 2. NY Pizza Suprema · 3. See No Evil Pizza · 4. Prince Street Pizza · 5. B Side Pizza & Wine Bar.",
  'score': 0.8051581},
 {'title': 'Manhattan Pizza - Home of Legendary, Manhattan-Style Pizza',
  'url': 'https://manhattanpizza.com/',
  'content': 'You will find a bit of Italy and Greece when you stop by Manhattan Pizza. This is the place to go for pizza, calzones, gyros, pasta, salads,',
  'score': 0.79675186},
 {'title': 'Don Antonio | Authentic Neapolitan Pizza in New York',
  'url': 'https://www.donantoniopizza.com/',
  'content': "Located in the heart of Hell's Kitchen and the Theatre District, Don Antonio brings the authentic flavors of Neapolitan pizza to Midtown Manhattan.",
  'score': 0.76596165},
 {'title': 'Best Pizza in New York City - Eater NY',
  'url': 'https://ny.eat

In [8]:
from dotenv import load_dotenv
_ = load_dotenv()

In [104]:
from typing import TypedDict, Annotated, Tuple, Dict, List
from langgraph.graph.message import add_messages
from langchain.schema import Document
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
  messages: Annotated[list, add_messages]
  search_query: str
  context: List[Document]
  search_results:Tuple[Union[List[Dict[str, str]], str], Dict]

In [105]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

EMBED_MODEL_URL = "https://klnki3w1q88gr09t.us-east-1.aws.endpoints.huggingface.cloud"

embeddings = HuggingFaceEndpointEmbeddings(
    model=EMBED_MODEL_URL,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
    )

In [106]:
client = QdrantClient(
    url=os.environ.get('QDRANT_DB_BITTER_MAMMAL'), # Name of the qdrant cluster is bitter_mammal
    api_key=os.environ.get('QDRANT_API_KEY_BITTER_MAMMAL'),
)
vector_store = QdrantVectorStore(
    client=client,
    collection_name="yelp_reviews",
    embedding=embeddings,
)
qdrant_retriever = vector_store.as_retriever()

/Users/deman/Dev/Maven/MavenAIBootcamp6/AIE6-Certification-Challenge/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


In [107]:
# RAG Prompt
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [108]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4.1-mini")

In [109]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | rag_prompt | openai_chat_model | StrOutputParser()
)

In [110]:
ASSISTANT_PROMPT = """
You are a foodie assistant. You can answer user's questions about food and restaurants.
The user may occasionally ask you questions followed by a context which you can rely on to
answer the questions. If the user has provided context, then use it to answer the question.

If the question is not related to food or restaurants, you can explain your purpose to the user.
If the user insists on an unrelated question, you must say "I'm just a foodie-assistant, I can't answer that."
"""

USER_PROMPT = """
{question}

{context}
"""

In [111]:
ROUTER_PROMPT = """
You are an intelligent router. You are given below a conversation between a user and an assistant.
Analyze the conversation and answer based on the following instructions:
1.Decide if the user's latest message needs additional context gathered from the Internet 
or a database or restuarant names, locations and reviews. If so, answer ONLY with a SINGLE WORD: "CONTEXT"
2. Otherwise, answer ONLY with the SINGLE WORD: "ASSISTANT"

Here is the conversation between the user and the assistant:

{messages}
"""

router_prompt = ChatPromptTemplate.from_template(ROUTER_PROMPT)

In [112]:
SEARCH_FORMULATOR_PROMPT = """
You are an expert at formulating search queries. You are given a history of interactions between
a user and an AI assistant. Forcus on the most recent user messages and formulate an independent search query
which can help answer the user's question. Respond ONLY with the search query text.

Here is the conversation between the user and the assistant:

{messages}
"""
search_formulator_prompt = ChatPromptTemplate.from_template(SEARCH_FORMULATOR_PROMPT)


In [113]:
def router(state: AgentState):
    chat_model = ChatOpenAI(model="gpt-4.1-mini")
    router_chain = router_prompt | chat_model | StrOutputParser()
    router_answer = router_chain.invoke(state["messages"])
    return router_answer

In [114]:
def search_formulator(state: AgentState):
    chat_model = ChatOpenAI(model="gpt-4.1-mini")
    search_formulator_chain = search_formulator_prompt | chat_model | StrOutputParser()
    search_query = search_formulator_chain.invoke(state["messages"])
    state["search_query"] = search_query
    return state

In [115]:
def search_engine(state: AgentState):
    search_query = state.get("search_query", "")
    if search_query:
        search_tool = TavilySearchResults(max_results=3)
        search_results = search_tool.invoke(search_query)
        state["search_results"] = search_results
    return state

In [116]:
def context_retriever(state: AgentState):
    search_query = state.get("search_query", "")
    if search_query:
        results = rag_chain.invoke({"question": search_query})
        state["context"] = results
    return state

In [117]:
def assistant(state: AgentState):
    chat_model = ChatOpenAI(model="gpt-4.1-mini")
    assistant_chain = chat_model
    latest_message = state["messages"][-1]
    latest_message = USER_PROMPT.format(question=latest_message.content, context=str(state["context"]) + str(state["search_results"]))
    state["messages"][-1] = latest_message
    # Pass a dictionary with the messages
    response = assistant_chain.invoke(state["messages"])
    return {"messages": [response]}
    

In [118]:
graph = StateGraph(AgentState)

graph.add_node("search_formulator", search_formulator)
graph.add_node("search_engine", search_engine)
graph.add_node("context_retriever", context_retriever)
graph.add_node("assistant", assistant)

graph.add_conditional_edges(START, router, {
    "CONTEXT": "search_formulator",
    "ASSISTANT": "assistant"
})

graph.add_edge("search_formulator", "search_engine")
graph.add_edge("search_engine", "context_retriever")
graph.add_edge("context_retriever", "assistant")
graph.add_edge("assistant", END)

graph = graph.compile()

In [119]:
messages = [
    AIMessage(content=ASSISTANT_PROMPT)
]
state_ = None

In [120]:
async def invoke_graph(query:str, graph: StateGraph):
    messages.append(HumanMessage(content=query))
    global state_
    state = AgentState(messages=messages, context=[], search_results=[])
    state_ = state
    async for chunk in graph.astream(state, stream_mode="updates"):
        for node, values in chunk.items():
            if node == "assistant":
                messages.append(values["messages"][-1])
            elif node == "search_formulator":
                state_["search_query"] = values["search_query"]
            elif node == "search_engine":
                state_["search_results"] = values["search_results"]
            elif node == "context_retriever":
                state_["context"] = values["context"]
            outp = f"Receiving update from node: '{node}'"
            state_ = state
            yield outp

In [121]:
async for outp in invoke_graph("What is the best pizza in New York?", graph):
    print(outp)

Receiving update from node: 'search_formulator'
Receiving update from node: 'search_engine'


/Users/deman/Dev/Maven/MavenAIBootcamp6/AIE6-Certification-Challenge/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Receiving update from node: 'context_retriever'
Receiving update from node: 'assistant'


In [122]:
state_

{'messages': [AIMessage(content='\nYou are a foodie assistant. You can answer user\'s questions about food and restaurants.\nThe user may occasionally ask you questions followed by a context which you can rely on to\nanswer the questions. If the user has provided context, then use it to answer the question.\n\nIf the question is not related to food or restaurants, you can explain your purpose to the user.\nIf the user insists on an unrelated question, you must say "I\'m just a foodie-assistant, I can\'t answer that."\n', additional_kwargs={}, response_metadata={}, id='a1920913-8d07-4562-947f-11200b3f9cf5'),
  HumanMessage(content='What is the best pizza in New York?', additional_kwargs={}, response_metadata={}, id='74873c3e-c5ba-4af1-9fdc-d6b1cccd5e76'),
  AIMessage(content='Based on recent reviews and discussions from 2024, some of the best pizza options in New York City for authentic and highly-rated pizza include:\n\n- **Joe’s Pizza (Greenwich Village and Times Square locations):** 

In [123]:
print(messages[-1].content)

Based on recent reviews and discussions from 2024, some of the best pizza options in New York City for authentic and highly-rated pizza include:

- **Joe’s Pizza (Greenwich Village and Times Square locations):** Famous for classic New York-style slices, especially at the original spot on 7 Carmine Street.
- **John’s Pizza (Bleeker Street):** A longstanding Village institution since 1929, well-regarded for whole pies.
- **Don Antonio (Midtown):** Known for Neapolitan-style pizza with a puffy crust, a favorite near West 50th & 8th Ave.
- **Luigi’s (Brooklyn):** Rated around 9.3/10 by some first-timers, considered among the best pizza tried in NYC.
- **Cafe Europa (Bensonhurst, Brooklyn):** Noted for its wood-fired oven and fresh, homemade ingredients.
- **Best Pizza (Williamsburg):** Mentioned as a standout spot that might make many top lists.

Also, a place simply called "New York Pizza" was noted for good New York-style qualities, indoor dining, and wine service, though opinions there 

In [124]:
async for outp in invoke_graph("What are some reviews for Di Fara's Pizza?", graph):
    print(outp)

Receiving update from node: 'search_formulator'
Receiving update from node: 'search_engine'


/Users/deman/Dev/Maven/MavenAIBootcamp6/AIE6-Certification-Challenge/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Receiving update from node: 'context_retriever'
Receiving update from node: 'assistant'


In [125]:
print(messages[-1].content)

Here are some recent reviews for Di Fara's Pizza in Brooklyn:

- One reviewer mentioned that Di Fara’s has a great New York-style pizza with a thick and crunchy crust that is somewhat cracker-like. Although NY style isn’t their favorite generally, they really enjoyed this pizza.  
- Another reviewer on Yelp called it “amazing” and claimed it’s the best square pizza they’ve found in NYC. They noted the chunky and very flavorful sauce, and said it’s a bit messy to eat but definitely worth it.  
- A third review praised the pepperoni pizza as the best in New York City according to their taste, highlighting the excellent herbs, spices, and sauce. They mentioned it’s pricey but worth it for the quality.

Overall, Di Fara's receives strong praise for its flavorful sauce, quality ingredients, and distinctive crust, making it a top choice for many pizza lovers in NYC. Would you like me to help find the best time to visit or recommendations for specific pies there?


In [126]:
async for outp in invoke_graph("What's the best coffee shop in Queens? What are some reveiws for it?", graph):
    print(outp)

Receiving update from node: 'search_formulator'
Receiving update from node: 'search_engine'


/Users/deman/Dev/Maven/MavenAIBootcamp6/AIE6-Certification-Challenge/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Receiving update from node: 'context_retriever'
Receiving update from node: 'assistant'


In [127]:
print(messages[-1].content)

One of the highly recommended coffee shops in Queens is **Sweetleaf Coffee Roasters**. It has a solid reputation with good reviews for its carefully roasted coffee and cozy atmosphere.

Here are some highlights from reviews of Sweetleaf Coffee Roasters in Queens:

- Customers often praise the quality and flavor of the coffee, noting that it’s fresh and expertly brewed.
- The vibe is described as warm and inviting, making it a great spot to relax or work.
- Some reviewers highlight the friendly and knowledgeable staff who help create a welcoming experience.

If you want more options, other notable coffee places in Queens include **Il Primo Cafe**, **Cafe Auburndale**, and **The Well Coffee**, each with positive reviews on Yelp.

Would you like recommendations tailored to a specific neighborhood in Queens or for a particular coffee style?
